In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Results - Generate TMAP

> Cluster 50k reaction data set by Schneider et al. using TMAP

In [ ]:
# hide
import json
import numpy as np
import pandas as pd
import tmap as tm
from tqdm import tqdm 
from faerun import Faerun
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from scipy import stats
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Descriptors3D
from matplotlib.colors import LinearSegmentedColormap


### Load data and generate minhash fingerprints

In [ ]:
lf = tm.LSHForest(256, 128)
mh_encoder = tm.Minhash()

In [ ]:
#hide
# data
with open('../data/rxnclass2name.json', 'r') as f:
    rxnclass2name = json.load(f)
schneider_df = pd.read_csv('../data/schneider50k.tsv', sep='\t', index_col=0)
ft_10k_fps = np.load('../data/fps_ft_10k.npz')['fps']
schneider_df['rxn_category'] = schneider_df.rxn_class.apply(lambda x: '.'.join(x.split('.')[:2]))
schneider_df['rxn_superclass'] = schneider_df.rxn_class.apply(lambda x: x.split('.')[0])
schneider_df.head()

,original_rxn,rxn_class,source,rxn,split,rxn_category,rxn_superclass
0,[CH3:17][S:14](=[O:15])(=[O:16])[N:11]1[CH2:10...,6.1.5,US06887874,C1CCCCC1.CCO.CS(=O)(=O)N1CCN(Cc2ccccc2)CC1.[OH...,test,6.1,6
1,O.O.[Na+].[CH3:1][c:2]1[cH:7][c:6]([N+:8](=O)[...,7.1.1,US07056926,CCOC(C)=O.Cc1cc([N+](=O)[O-])ccc1NC(=O)c1ccccc...,test,7.1,7
2,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6](-[c:9]2[cH...,1.8.5,US08492378,COc1ccc(-c2coc3ccc(-c4nnc(S)o4)cc23)cc1.COc1cc...,test,1.8,1
3,Cl.[CH3:43][CH2:42][S:44](=[O:45])(=[O:46])Cl....,2.2.3,US08592454,CCS(=O)(=O)Cl.CN(C(=O)N(C)[C@@H]1CN(C(=O)C2CCN...,train,2.2,2
4,[CH3:25][O:24][c:21]1[cH:22][cH:23][c:17]([O:1...,1.3.7,US06716851,COc1ccc(OC)c(N)c1.Cc1cc(Cl)nc(-c2ccccn2)n1>>CO...,test,1.3,1


In [ ]:
# slow
mhfps = [mh_encoder.from_weight_array(fp.tolist(), method="I2CWS") for fp in tqdm(ft_10k_fps)]

100%|██████████| 50000/50000 [00:15<00:00, 3213.43it/s]



### Calculate rxn properties

In [ ]:
# slow

labels = []
# superclasses
superclasses = []

# product properties
tpsa = []
logp = []
mw = []
h_acceptors = []
h_donors = []
ring_count = []

# metals in precursors
has_Pd = []
has_Li = []
has_Mg = []
has_Al = []

for i, row in tqdm(schneider_df.iterrows(), total=len(schneider_df)):

    rxn = row["rxn"]
    labels.append(
        str(rxn)
        + "__"
        + str(rxn)
        + f"__{row['source']}"
        + f"__{rxnclass2name[row['rxn_class']]} - {row['rxn_class']}"
        + f"__{rxnclass2name[row['rxn_category']]}"
        + f"__{rxnclass2name[row['rxn_superclass']]}"
    )
    superclasses.append(int(row["rxn_superclass"]))
    
    precursors, products = rxn.split('>>')

    mol = Chem.MolFromSmiles(products)
            
    tpsa.append(Descriptors.TPSA(mol))
    logp.append(Descriptors.MolLogP(mol))
    mw.append(Descriptors.MolWt(mol))
    h_acceptors.append(Descriptors.NumHAcceptors(mol))
    h_donors.append(Descriptors.NumHDonors(mol))
    ring_count.append(Descriptors.RingCount(mol))
    
    has_Pd.append('Pd' in precursors)
    has_Li.append('Li' in precursors)
    has_Mg.append('Mg' in precursors)
    has_Al.append('Al' in precursors)
tpsa_ranked = stats.rankdata(np.array(tpsa) / max(tpsa)) / len(tpsa)
logp_ranked = stats.rankdata(np.array(logp) / max(logp)) / len(logp)
mw_ranked = stats.rankdata(np.array(mw) / max(mw)) / len(mw)
h_acceptors_ranked = stats.rankdata(np.array(h_acceptors) / max(h_acceptors)) / len(
    h_acceptors
)
h_donors_ranked = stats.rankdata(np.array(h_donors) / max(h_donors)) / len(h_donors)
ring_count_ranked = stats.rankdata(np.array(ring_count) / max(ring_count)) / len(
    ring_count
)
labels_groups, groups = Faerun.create_categories(superclasses)

labels_groups = [(label[0], f"{label[1]} - {rxnclass2name[str(label[1])]}") for label in labels_groups]


100%|██████████| 50000/50000 [00:50<00:00, 987.99it/s] 


### Configure LSH forest 

In [ ]:
# slow
lf.batch_add(mhfps)
lf.index()

# Layout
cfg = tm.LayoutConfiguration()
cfg.k = 50
cfg.kc = 50
cfg.sl_scaling_min = 1.0
cfg.sl_scaling_max = 1.0
cfg.sl_repeats = 1
cfg.sl_extra_scaling_steps = 2
cfg.placer = tm.Placer.Barycenter
cfg.merger = tm.Merger.LocalBiconnected
cfg.merger_factor = 2.0
cfg.merger_adjustment = 0
cfg.fme_iterations = 1000
cfg.sl_scaling_type = tm.ScalingType.RelativeToDesiredLength
cfg.node_size = 1 / 37
cfg.mmm_repeats = 1

# Define colormaps
set1 = plt.get_cmap("Set1").colors
rainbow = plt.get_cmap("rainbow")
colors = rainbow(np.linspace(0, 1, len(set(groups))))[:, :3].tolist()
custom_cm = LinearSegmentedColormap.from_list("my_map", colors, N=len(colors))
bin_cmap = ListedColormap([set1[8], "#5400F6"], name="bin_cmap")

# Get tree coordinates
x, y, s, t, _ = tm.layout_from_lsh_forest(lf, config=cfg)

### Create Fearun plot 

In [ ]:
# slow
f = Faerun(clear_color="#ffffff", coords=False, view="front",)
    
f.add_scatter(
"ReactionAtlas",
{
    "x": x, "y": y, 
    "c": [
        groups, # superclasses
        has_Pd, 
        has_Li, 
        has_Mg, 
        has_Al,
        tpsa_ranked,
        logp_ranked,
        mw_ranked,
        h_acceptors_ranked,
        h_donors_ranked,
        ring_count_ranked,
    ], 
    "labels": labels
},
shader="smoothCircle",
colormap=[
    custom_cm, 
    bin_cmap, 
    bin_cmap, 
    bin_cmap, 
    bin_cmap, 
    "rainbow",
    "rainbow",
    "rainbow",
    "rainbow",
    "rainbow",
    "rainbow",

],
point_scale=2.0,
categorical=[
    True, 
    True, 
    True, 
    True, 
    True, 
    False, 
    False, 
    False, 
    False, 
    False, 
    False, 
],
has_legend=True,
legend_labels=[
    labels_groups,
    [(0, "No"), (1, "Yes")],
    [(0, "No"), (1, "Yes")],
    [(0, "No"), (1, "Yes")],
    [(0, "No"), (1, "Yes")],
    None,
    None,
    None,
    None,
    None,
    None,
],
selected_labels=["SMILES", "SMILES", "Patent ID",  "Named Reaction", "Category", "Superclass"],
series_title=[
    "Superclass", 
    "Pd", 
    "Li", 
    "Mg", 
    "Al",
    "TPSA",
    "logP",
    "Mol Weight",
    "H Acceptors",
    "H Donors",
    "Ring Count",
],
max_legend_label=[
    None,
    None,
    None,
    None,
    None,
    str(round(max(tpsa))),
    str(round(max(logp))),
    str(round(max(mw))),
    str(round(max(h_acceptors))),
    str(round(max(h_donors))),
    str(round(max(ring_count))),
],
min_legend_label=[
    None,
    None,
    None,
    None,
    None,
    str(round(min(tpsa))),
    str(round(min(logp))),
    str(round(min(mw))),
    str(round(min(h_acceptors))),
    str(round(min(h_donors))),
    str(round(min(ring_count))),
],
title_index=2,
legend_title="",
)

f.add_tree("reactiontree", {"from": s, "to": t}, point_helper="ReactionAtlas")

In [ ]:
# hide
# slow
plot = f.plot("ft_10k_schneider_50k", template="reaction_smiles")

/Users/phs/git/colab/bert/rxnfp/rxnfp_release/nbs/ft_10k_schneider_50k.html

### Result

The result of running `f.plot("ft_10k_schneider_50k", template="reaction_smiles")` is:



<div style="text-align: center">
<img src='images/sample_tmap.jpg' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reaction atlas of 50k Schneider data set. Product and precursor properties are highlighted in the different layers. </p>
</div>

### Interative version

An interactive reaction atlas made from the same data set and fingerprint can be found here (**[link](../tmaps/tmap_ft_10k.html)**).